<a href="https://colab.research.google.com/github/screid/Estadistica_Computacional_UGM/blob/main/Clase_12_Modelos_Avanzados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.formula.api import mixedlm
%config InlineBackend.figure_format = 'retina'

## 1 · Regresión Logística (Binomial GLM)

Usaremos la versión tabular del famoso dataset del *Titanic*.  
La variable `Survived` (0 = No, 1 = Sí) será la respuesta.

## 1.1. Dataset y modelo

In [3]:
# 1. Cargar y “aplanar” el Titanic
titanic = sm.datasets.get_rdataset('Titanic', package='datasets').data
titanic = titanic.loc[titanic.index.repeat(titanic['Freq'])].reset_index(drop=True)

In [8]:
titanic.head()

,Class,Sex,Age,Survived,Freq
0,3,1,0,0,35
1,3,1,0,0,35
2,3,1,0,0,35
3,3,1,0,0,35
4,3,1,0,0,35


In [4]:
# 2. Codificar variables (pasarlas de numéricas a categóricas)
titanic['Survived'] = (titanic['Survived'] == 'Yes').astype(int)
titanic['Sex']      = (titanic['Sex'] == 'Male').astype(int)
titanic['Age']      = (titanic['Age'] == 'Adult').astype(int)
titanic['Class']    = titanic['Class'].map({'1st': 1, '2nd': 2, '3rd': 3, 'Crew': 4})

In [5]:
# 3. Matrices para el GLM
X = sm.add_constant(titanic[['Sex', 'Age', 'Class']].astype(float))
y = titanic['Survived']

logit_model = sm.GLM(y, X, family=sm.families.Binomial()).fit()
print(logit_model.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:               Survived   No. Observations:                 2201
Model:                            GLM   Df Residuals:                     2197
Model Family:                Binomial   Df Model:                            3
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1149.6
Date:                Tue, 27 May 2025   Deviance:                       2299.2
Time:                        22:21:28   Pearson chi2:                 2.22e+03
No. Iterations:                     4   Pseudo R-squ. (CS):             0.1924
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.0990      0.255      8.218      0.0

## 1.2. Interpretación de resultados

| Término             | Coef. (β) | **exp(β)** | 95 % IC de exp(β) | p-valor | Lectura práctica                                                                                          |
| ------------------- | --------- | ---------- | ----------------- | ------- | --------------------------------------------------------------------------------------------------------- |
| **const**           | +2.099    | **8.16**   | 4.95 – 13.46      | <0.001  | Basándose en una mujer, niña, de 1ª clase, la probabilidad base de sobrevivir es alta (odds ≈ 8 : 1).     |
| **Sex (Male = 1)**  | –2.058    | **0.13**   | 0.10 – 0.16       | <0.001  | Ser hombre reduce las odds de supervivencia ≈ 87 % (1 – 0.13).                                            |
| **Age (Adult = 1)** | –0.512    | **0.60**   | 0.39 – 0.93       | 0.022   | Ser adulto disminuye las odds de sobrevivir ≈ 40 % frente a un niño, manteniendo todo lo demás constante. |
| **Class**           | –0.278    | **0.76**   | 0.69 – 0.84       | <0.001  | Cada nivel “más bajo” de clase (1→2, 2→3, 3→Crew) reduce las odds ≈ 24 %.                                 |


Detalle interpretación

* exp(β) da el odds ratio:

  * < 1 ⇒ factor protector (odds bajan), > 1 ⇒ factor de riesgo (odds suben).

* El efecto de sexo es el más fuerte; ser hombre multiplica las odds por 0.13.

* El pseudo-R² de Cox-Snell (0.19) indica que el modelo explica ~19 % de la variabilidad—razonable para datos de supervivencia.

* Todos los coeficientes son estadísticamente significativos (p < 0.05).

* Ejemplo numérico
  ** Para un hombre adulto de 3ª clase:

  $log_{odds} = 2.099-2.058(1) - 0.512(1) - 0.278(3) = - 1.285$

Odds = $e^{-1.285} ≈ 0.28$ → Probabilidad ≈ 0.22 (22 %).

**Nota 1:** El modelo resultante sería: $log_{odds} = \beta_0 + \beta_{sex} x_{sex} + \beta_{age} x_{age} + \beta_{class} x_{class}$

**Nota 2:** const representa la log-odds de supervivencia cuando Sex = 0 (mujer), Age = 0 (niña) y Class = 0 (una categoría fuera de los datos). De igual manera, β0 (el const) es la log-odds de sobrevivir cuando todas las variables explicativas valen 0.

## 2 · Regresión de Poisson (conteos)

## 2.1. Dataset y modelo

Generaremos un ejemplo artificial: número de llamadas recibidas por hora, con un indicador `hora_pico` (0/1).

In [7]:
# 1. Creo Dataset fictio
np.random.seed(42)
n = 200
df_calls = pd.DataFrame({
    'llamadas': np.random.poisson(lam=3, size=n) + np.random.binomial(1, 0.3, size=n),
    'hora_pico': np.random.binomial(1, 0.4, size=n)
})

# 2. Ejecuto modelo
Xc = sm.add_constant(df_calls[['hora_pico']])
pois_model = sm.GLM(df_calls['llamadas'], Xc, family=sm.families.Poisson()).fit()
pois_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:               llamadas   No. Observations:                  200
Model:                            GLM   Df Residuals:                      198
Model Family:                 Poisson   Df Model:                            1
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -400.76
Date:                Tue, 27 May 2025   Deviance:                       245.49
Time:                        22:33:42   Pearson chi2:                     222.
No. Iterations:                     4   Pseudo R-squ. (CS):           0.004345
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.1736      0.054     21.830      0.000       1.068       1.279
hora_pico     -0.0750      0.080     -0.932      0.351      -0.233       0.083
==============================================================================
"""

## 2.2. Interpretación de resultados

| Parámetro      | β (log-coef) | **IRR = exp(β)** | IC 95 % de IRR | p -valor | Lectura práctica                                                                                                                |
| -------------- | -----------: | ---------------: | -------------: | -------: | ------------------------------------------------------------------------------------------------------------------------------- |
| **const**      |       +1.174 |         **3.23** |    2.90 – 3.60 |  < 0.001 | En hora **no pico** ( `hora_pico = 0` ) se esperan ≈ 3.23 llamadas en promedio.                                                 |
| **hora\_pico** |       –0.075 |         **0.93** |    0.79 – 1.09 |    0.351 | En hora **pico** las llamadas son un **7 % menores** (IRR ≈ 0.93), pero la diferencia **no es estadísticamente significativa**. |


Qué significan estos números
* Incident Rate Ratio (IRR)

  * IRR > 1 → aumento de la tasa de llamadas.

  * IRR < 1 → disminución.

  * Aquí, 0.93 indica un posible 7 % menos, pero el p-valor = 0.35 > 0.05 → no podemos afirmar que exista efecto.

* Bondad de ajuste

  * Pseudo R² (Cox-Snell) = 0.004 → el modelo apenas explica varianza: coherente con un único predictor débil.

  * Deviance / df ≈ 245.5 / 198 = 1.24 (> 1) ⇒ sobredispersión ligera; si fuera real, se consideraría una familia Quasi-Poisson o NegBin.

En este ejemplo artificial, hora_pico no afecta significativamente el número de llamadas.

Si en otro análisis se obtuviera, por ejemplo, β = 0.223 (IRR ≈ 1.25) con IC 1.10 – 1.42 y p < 0.001, entonces sí podrías decir con confianza: “en hora pico las llamadas aumentan un 25 %”.

## 3 · Modelo Mixto Lineal (LMM)

## 3.1. Cargar dataset y ejecutar modelo

Dataset *sleepstudy* (tiempo de reacción vs días sin dormir) con interceptos aleatorios por sujeto.

In [9]:
# 1. Cargar dataset
sleep = sm.datasets.get_rdataset('sleepstudy', 'lme4').data

#. Ejecutar modelo
lmm = mixedlm('Reaction ~ Days', data=sleep, groups=sleep['Subject']).fit()
lmm.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
         Mixed Linear Model Regression Results
========================================================
Model:             MixedLM Dependent Variable: Reaction 
No. Observations:  180     Method:             REML     
No. Groups:        18      Scale:              960.4568 
Min. group size:   10      Log-Likelihood:     -893.2325
Max. group size:   10      Converged:          Yes      
Mean group size:   10.0                                 
--------------------------------------------------------
           Coef.   Std.Err.   z    P>|z|  [0.025  0.975]
--------------------------------------------------------
Intercept  251.405    9.747 25.794 0.000 232.302 270.508
Days        10.467    0.804 13.015 0.000   8.891  12.044
Group Var 1378.176   17.156                             
========================================================

"""

## 3.2. Interpretación

| Componente                          |             Valor | p-valor | Lo que significa                                                                                                                  |
| ----------------------------------- | ----------------: | :-----: | --------------------------------------------------------------------------------------------------------------------------------- |
| **Intercepto fijo**                 |      **251.4 ms** |  <0.001 | Tiempo medio de reacción cuando Days = 0 para un **sujeto “promedio”**.                                                           |
| **Pendiente fija (`Days`)**         | **+10.47 ms/día** |  <0.001 | Cada día adicional sin dormir incrementa el tiempo de reacción en ≈ 10.5 ms, en promedio. **Efecto fuerte y significativo.**      |
| **Varianza interceptos aleatorios** |           1 378.2 |    —    | Diferencias entre sujetos: SD ≈ √1 378 ≈ **37 ms**. Algunos parten bastante más lentos o más rápidos que el “promedio de 251 ms”. |
| **Varianza residual (Scale)**       |             960.5 |    —    | Variabilidad **dentro** de cada sujeto tras ajustar la pendiente: SD ≈ √960 ≈ **31 ms**.                                          |


** Lectura práctica**

* Efecto diario

  * Tras 5 días sin dormir, se espera que la reacción sea ~52 ms más lenta (5*10.47) respecto al día 0.

* Heterogeneidad entre personas

  * Un sujeto con un intercepto +1 SD (+37 ms) ya parte en ≈288 ms.

  * Otro con -1 SD (-37 ms) parte en ≈214 ms.

  * Esta dispersión explica 59 % de la varianza total

  $ICC = 1378/(1378+960) \approx 0.59$

* Precisión

  * IC 95 % de la pendiente: 8.9 - 12.0 ms/día ⇒ el efecto es claramente distinto de 0.

  * Convergencia “Yes” -- el algoritmo encontró un óptimo estable.

Todos los sujetos se deterioran al mismo ritmo (~10 ms/día), pero no parten iguales: hay ~37 ms de variación inicial entre personas. El modelo mixto captura esa diferencia de línea base y evita atribuirla erróneamente al efecto de los días.